# AI Assignment from Vijayi WFH Technologies Pvt Ltd 

## Task 2 - RAG pipeline implementation

In [1]:
import pandas as pd
import faiss
import json
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import numpy as np

In [2]:
# Loading data and model
df = pd.read_csv('processed_quotes.csv')  # Saved from previous step
model = SentenceTransformer('./fine_tuned_model')
generator = pipeline('text-generation', model='gpt2-medium')

# Creating quote embeddings
quote_embeddings = model.encode(df['processed_quote'].tolist(), show_progress_bar=True)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

C:\Users\HP\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

In [5]:
# Building FAISS index
dimension = quote_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(quote_embeddings.astype('float32'))

# Retrieval function
def retrieve_quotes(query, k=5):
    query_embed = model.encode([query])
    distances, indices = index.search(query_embed, k)
    
    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            'quote': df.iloc[idx]['quote'],
            'author': df.iloc[idx]['author'],
            'tags': df.iloc[idx]['tags'],
            'score': float(1/(1 + distances[0][i])) }) # Convert distance to similarity
    return results

In [6]:
# RAG response generation function
def generate_response(query, context_quotes):
    context = "\n".join([f"{q+1}. {item['quote']} - {item['author']}" 
                        for q, item in enumerate(context_quotes)])
    
    prompt = f"""Context quotes:{context} Question: {query} Answer:"""
    
    response = generator(
        prompt,
        max_length=256,
        num_return_sequences=1,
        temperature=0.7
    )
    return response[0]['generated_text'].split("Answer:")[-1].strip()

In [7]:
# Full RAG pipeline
def rag_pipeline(query):
    retrieved = retrieve_quotes(query)
    answer = generate_response(query, retrieved)
    return {
        "query": query,
        "answer": answer,
        "source_quotes": retrieved
    }

In [8]:
# Save components for later use
faiss.write_index(index, "quote_index.faiss")
df.to_pickle("quotes_df.pkl")